In [0]:
import sys
from os.path import isfile
from IPython.core.magic import Magics, cell_magic, magics_class
from IPython.core.magic_arguments import argument, magic_arguments, parse_argstring

@magics_class
class GPPPlugin(Magics):
    @staticmethod
    def write_file(name, contents):
        with open(name, "w") as f:
            f.write(contents)

    @staticmethod
    def compile_app():
        !rm -f main.out
        GPPPlugin.compile_source('main')
        !g++ *.o -Wall -o main.out

    @staticmethod
    def compile_source(name):
        source_name = f'{name}.cpp'
        object_name = f'{name}.o'
        !rm -f $object_name
        !g++ -c $source_name -Wall
    
    @staticmethod
    def run():
        if isfile('main.out'):
            !./main.out

    @magic_arguments()
    @argument('-n', '--name', type=str, help='File name to be produced by the cell (without extension)')
    @cell_magic
    def gpp_header(self, line, cell):
        args = parse_argstring(self.gpp_header, line)
        if not args.name:
            print("Must provide a name for the resulting file (without extension)")
            return
        self.write_file(f'{args.name}.h', cell)


    @magic_arguments()
    @argument('-n', '--name', type=str, help='File name to be produced by the cell (without extension)')
    @cell_magic
    def gpp_source(self, line, cell):
        args = parse_argstring(self.gpp_source, line)
        if not args.name:
            print("Must provide a name for the resulting file (without extension)")
            return
        self.write_file(f'{args.name}.cpp', cell)
        self.compile_source(args.name)

    @cell_magic
    def gpp_main(self, line, cell):
        self.write_file('main.cpp', cell)
        self.compile_app()
        self.run()

get_ipython().register_magics(GPPPlugin)

In [0]:
%%gpp_header --name welcomer

#ifndef WELCOMER
#define WELCOMER

#include <string>
#include <iostream>

class Welcomer
{
    std::string name;

public:
    Welcomer(std::string name);
    std::string get_welcome();
};

#endif

In [0]:
%%gpp_source --name welcomer

#include "welcomer.h"


Welcomer::Welcomer(std::string name) {
    this->name = name;
}
std::string Welcomer::get_welcome() {
    return "Hello " + this->name;
}

In [10]:
%%gpp_main

#include <iostream>
#include "welcomer.h"


int main(void){
    Welcomer w("Ash");
    std::cout << w.get_welcome() << std::endl;
    return 0;
}

Hello Ash
